In [1]:
# pip install opencv-python 설치 필요
# tensorflow ver 2.16.1
# keras ver 3.3.3
import cv2
import os
import math
import time
from PIL import Image
import numpy as np
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
# 폭력이 감지된 시점부터 10초의 이미지를 저장 / 10fps 당 1장을 추출
# 매번 터치 생길때마다 10초씩 추출 / 단 중복은 허용해서 저장하지 않음 
# 폭력이 감지된 시점부터 10초의 이미지를 저장 / 10fps 당 1장을 추출
# 매번 터치 생길때마다 10초씩 추출 / 단 중복은 허용해서 저장하지 않음 

In [2]:
# 폴더 속 파일 목록을 모두 가져오는 function들

def read_folder_endswith_mp4(folder_path) : # mp4 파일 목록 반환
    file_list_all = os.listdir(folder_path)
    file_list = [file for file in file_list_all if file.endswith('mp4')]
    return file_list

# def read_folder_endswith_jpg(folder_path) : # jpg 파일 목록 반환
#     file_list_all = os.listdir(folder_path)
#     file_list = [file for file in file_list_all if file.endswith('jpg')]
#     return file_list

# def read_folder_startswith(folder_path, str) : #str로 시작하는 파일 목록 반환
#     file_list_all = os.listdir(folder_path)
#     file_list = [file for file in file_list_all if file.startswith(str)]
#     return file_list

# def get_file_name(file_list) : # 확장자를 제외한 파일 목록을 반환
#     video_file_list = [file[:-4] for file in file_list]
#     return video_file_list


In [3]:
def auto_extract_frame(video_name, video_path, output_path, frame_rate, s_list) :

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("auto_extract_frame에서 비디오 파일을 열 수 없습니다.")
        return

    # FPS(Frame Per Second) 정보 가져오기
    fps = math.ceil(cap.get(cv2.CAP_PROP_FPS))

    for i in range(len(s_list)) : # 접촉 시점마다 전후 1초동안 7장의 이미지를 추출

        start_time = s_list[i]
        interval = 2

        # 원하는 시간의 프레임 번호 계산
        frame_number = int((start_time- 1) * fps)
        # frame_number = int(start_frame - fps)

        end_frame_number = frame_number + interval * fps + 1

        # # 영상의 끝지점의 경우 7장의 이미지 추출이 어려울 수 있으므로 사진을 저장하지 않음
        # cap.set(cv2.CAP_PROP_POS_FRAMES, end_frame_number - 1)
        # ret, frame = cap.read()
        # if ret != True :
        #     break
        
        while(frame_number <= end_frame_number) :
             
            # 해당 프레임으로 위치 설정
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    
            # 프레임 읽기
            ret, frame = cap.read()
        
            if ret:
                # 프레임을 이미지 파일로 저장
                # print("frame_number : ")
                # print(frame_number )
                image_output_path = f"{output_path}/{video_name[:-4]}_startTime{start_time}_{frame_number}_CA_Y.jpg"
                
                cv2.imwrite(image_output_path, frame)
                # print(f"{frame_number} 프레임이 성공적으로 저장되었습니다: {image_output_path}")
            else:
                print("프레임을 읽을 수 없습니다.")
                break
            
            frame_number += int(fps / frame_rate)
        
        i += 1

    # 비디오 캡처 객체 해제
    cap.release()


In [4]:
def touch_detection(video_name, video_path, output_path, start_time, frame_rate, model_path):

    print(f"{video_name}, 접촉 여부 판정이 시작되었습니다")
    
    # 모형 불러오기
    model = models.load_model(model_path) # model 불러오기

    # 비디오 파일 열기
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("touch_detection에서 비디오 파일을 열 수 없습니다.")
        return
    
    # FPS(Frame Per Second) 정보 가져오기
    fps = math.ceil(cap.get(cv2.CAP_PROP_FPS))
    
    if fps == 0:
        print("FPS 정보를 가져올 수 없습니다.")
        return
    
    # 원하는 시간의 프레임 번호 계산
    frame_number = int(start_time * fps)
    
    # frame_list = []
    touch_time = []
    while(True) : #동영상 끝까지 이미지를 추출
        
        # 해당 프레임으로 위치 설정
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

        # 프레임 읽기
        ret, frame = cap.read()
    
        if ret:
            # 프레임을 이미지 파일로 저장
            print(output_path)
            image_output_path = f"{output_path}/{video_name[:-4]}_{int(frame_number / fps)}.jpg"
            cv2.imwrite(image_output_path, frame)

            # 이미지 가공
            IMG_WIDTH, IMG_HEIGHT = 224, 224
            img = Image.open(image_output_path)
            img = img.resize((IMG_WIDTH, IMG_HEIGHT))
            np_img = image.img_to_array(img)
            img_batch = np.expand_dims(np_img, axis=0) #이미지 4차원으로 변형
            pre_processed = preprocess_input(img_batch) # nomalization

            # 모형을 통한 이미지의 접촉 여부 예측
            y_preds = model.predict(pre_processed)

            # 접촉 여부 확률에 따른 접촉 시간 저장
            if(y_preds[0][1] > 0.6) :

                end_frame_number = frame_number + fps 
                cap.set(cv2.CAP_PROP_POS_FRAMES, end_frame_number)
                ret, frame = cap.read()
                if ret != True :
                    os.remove(image_output_path)
                    break
                
                touch_time.append(int(frame_number / fps))
                # frame_list.append(frame_number)
                
                print(f"{int(frame_number / fps)}초에 접촉이 감지되었습니다.")
                print("접촉 일어났을 확률 :", y_preds[0][1])
            else :
                # print(f"{frame_number / fps}초에 접촉이 감지되지 않았습니다.")
                # print( y_preds)
                os.remove(image_output_path)
        else:
            print("프레임을 읽을 수 없습니다.")
            break

        frame_number += int(fps / frame_rate)
    
    # 비디오 캡처 객체 해제
    cap.release()
    return touch_time

In [21]:
def video_to_image_by_Model0(video_folder_path, output_path_check_touch, output_path_check_child_abuse, start_time) :
    

    # video_folder_path = './CA_Y_Videos'  # 현재폴더 기준
    
    # output_path_check_touch = './image_check_touch_by_Model0_CA_Y'  # 접촉 여부 판정한 이미지가 저장될 폴더
    frame_rate_check_touch = 1  # 1초당 1장을 저장
    
    # output_path_check_child_abuse = './CA_Y_Image'  # 접촉 여부 판정한 이미지가 저장될 폴더
    frame_rate_check_child_abuse = 3  # 1초당 3장


    model_path = "./model0_layer3_monitorLoss_Adam.keras"
    
    file_list = read_folder_endswith_mp4(video_folder_path)
    print(f"video list = {file_list}")
    print(f"총 영상의 개수는 {len(file_list)}개 입니다.")
    print("")
    
    i = 0
    for video_name in file_list : 
        i += 1
        video_path = video_folder_path + "/" + video_name
    
        print("")
        print(f"{i}번째 영상인")
        # if(i ==  10) :
        #     time.sleep(120)

        # if(i ==  30) :
        #     time.sleep(120)

        
        # if(i == 50) :
        #     time.sleep(60)
            
        touch_times = touch_detection(video_name, video_path, output_path_check_touch, start_time, frame_rate_check_touch, model_path)
        print(touch_times)
        auto_extract_frame(video_name, video_path, output_path_check_child_abuse, frame_rate_check_child_abuse, touch_times)
    
        print("")
        print(f"{i}번째 영상에 대한 학대장면 추출이 완료되었습니다.")
        print("")
    
    print("")
    print("#######################################")
    print(f"전체 영상에 대한 추출이 완료되었습니다.")
    print("#######################################")

In [8]:
video_to_image_by_Model0("./test/CA_Y_video", "./test/image_check_touch_by_Model0_CA_Y", "./test/CA_Y")

video list = ['C012_A02_SY04_P06_S01_02DAS.mp4', 'C012_A02_SY08_P03_S02_01DBS.mp4', 'C012_A02_SY09_P03_S01_02DBS.mp4', 'C012_A02_SY09_P07_S01_01DBS.mp4', 'C012_A03_SY07_P01_S42_01DAS.mp4', 'C012_A04_SY04_P06_S03_01NBS.mp4', 'C012_A04_SY04_P09_S01_03DBS.mp4', 'C012_A04_SY09_P06_S04_05DBS.mp4', 'C012_A04_SY09_P07_S03_01DAS.mp4', 'C012_A04_SY10_P01_S02_02DAS.mp4']
총 영상의 개수는 10개 입니다.


1번째 영상인
C012_A02_SY04_P06_S01_02DAS.mp4, 접촉 여부 판정이 시작되었습니다
./test/image_check_touch_by_Model0_CA_Y
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
./test/image_check_touch_by_Model0_CA_Y
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
./test/image_check_touch_by_Model0_CA_Y
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
./test/image_check_touch_by_Model0_CA_Y
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
./test/image_check_touch_by_Model0_CA_Y
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
./test/image_check_touch_by_Model0_CA_Y
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
65초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.8279082
./test/image_check_touch_by_Model0_CA_Y
1/1 ━━

In [15]:
video_to_image_by_Model0("./test/CA_N_video", "./test/image_check_touch_by_Model0_CA_N", "./test/CA_N",1)

video list = ['17.mp4', '2.mp4', 'extracted_segment_000042.mp4', 'extracted_segment_000122.mp4', 'extracted_segment_000205.mp4', 'extracted_segment_001155.mp4', 'extracted_segment_001306.mp4']
총 영상의 개수는 7개 입니다.


1번째 영상인
17.mp4, 접촉 여부 판정이 시작되었습니다
./test/image_check_touch_by_Model0_CA_N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.60894716
./test/image_check_touch_by_Model0_CA_N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
2초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.9366079
./test/image_check_touch_by_Model0_CA_N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
3초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.93490195
./test/image_check_touch_by_Model0_CA_N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
4초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.9491164
./test/image_check_touch_by_Model0_CA_N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
[1, 2, 3, 4]

1번째 영상에 대한 학대장면 추출이 완료되었습니다.


2번째 영상인
2.mp4, 접촉 여부 판정이 시작되었습니다
./test/image_check_touch_by_Model0_CA_N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.917715
./te

In [17]:
video_to_image_by_Model0("./valid/CA_N_video", "./valid/image_check_touch_by_Model0_CA_N", "./valid/CA_N",1)
video_to_image_by_Model0("./valid/CA_Y_video", "./valid/image_check_touch_by_Model0_CA_Y", "./valid/CA_Y", 60)

video list = ['11.mp4', '3.mp4', '9.mp4', 'extracted_segment_000010.mp4', 'extracted_segment_000015.mp4', 'extracted_segment_000057.mp4', 'extracted_segment_000310.mp4', 'extracted_segment_000450.mp4', 'extracted_segment_000917.mp4', 'extracted_segment_001315.mp4', 'extracted_segment_001339.mp4', 'extracted_segment_004334.mp4']
총 영상의 개수는 12개 입니다.


1번째 영상인
11.mp4, 접촉 여부 판정이 시작되었습니다
./valid/image_check_touch_by_Model0_CA_N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
1초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.7544732
./valid/image_check_touch_by_Model0_CA_N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
./valid/image_check_touch_by_Model0_CA_N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
3초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.84892756
./valid/image_check_touch_by_Model0_CA_N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
4초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.7177763
./valid/image_check_touch_by_Model0_CA_N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
./valid/image_check_touch_by_Model0_CA_N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
./valid/image_check

In [ ]:
video_to_image_by_Model0("./train/CA_N_video", "./train/image_check_touch_by_Model0_CA_N", "./train/CA_N",1)

video list = ['1.mp4', '15.mp4', '16.mp4', '18.mp4', '20.mp4', '4.mp4', '5.mp4', '6.mp4', '7.mp4', '8.mp4', 'extracted_segment_000000.mp4', 'extracted_segment_000000_1.mp4', 'extracted_segment_000000_2.mp4', 'extracted_segment_000000_3.mp4', 'extracted_segment_000003.mp4', 'extracted_segment_000016.mp4', 'extracted_segment_000019.mp4', 'extracted_segment_000025.mp4', 'extracted_segment_000028.mp4', 'extracted_segment_000043.mp4', 'extracted_segment_000049.mp4', 'extracted_segment_000052.mp4', 'extracted_segment_000107.mp4', 'extracted_segment_000108.mp4', 'extracted_segment_000120.mp4', 'extracted_segment_000149.mp4', 'extracted_segment_000150.mp4', 'extracted_segment_000200.mp4', 'extracted_segment_000207.mp4', 'extracted_segment_000320.mp4', 'extracted_segment_000335.mp4', 'extracted_segment_000340.mp4', 'extracted_segment_000403.mp4', 'extracted_segment_000414.mp4', 'extracted_segment_000638.mp4', 'extracted_segment_000721.mp4', 'extracted_segment_000929.mp4', 'extracted_segment_000

In [22]:
video_to_image_by_Model0("./train/CA_Y_video", "./train/image_check_touch_by_Model0_CA_Y", "./train/CA_Y", 60)

video list = ['C012_A02_SY04_P07_S01_01DBS.mp4', 'C012_A02_SY04_P07_S04_03DBS.mp4', 'C012_A02_SY04_P10_S01_01DBS.mp4', 'C012_A02_SY04_P10_S02_01DBS.mp4', 'C012_A02_SY07_P01_S03_01DAS.mp4', 'C012_A02_SY07_P01_S04_01DAS.mp4', 'C012_A02_SY07_P03_S01_01DBS.mp4', 'C012_A02_SY07_P06_S04_01DAS.mp4', 'C012_A02_SY08_P03_S01_01DBS.mp4', 'C012_A02_SY08_P03_S04_02DBS.mp4', 'C012_A02_SY08_P04_S01_01DAS.mp4', 'C012_A02_SY08_P05_S01_01DBS.mp4', 'C012_A02_SY08_P05_S02_02DBS.mp4', 'C012_A02_SY08_P06_S01_01DAS.mp4', 'C012_A02_SY09_P03_S02_02DAS.mp4', 'C012_A02_SY09_P04_S01_06DBS.mp4', 'C012_A02_SY09_P04_S02_02DAS.mp4', 'C012_A02_SY09_P05_S01_06DBS.mp4', 'C012_A02_SY09_P06_S01_05DAS.mp4', 'C012_A02_SY09_P06_S02_02DBS.mp4', 'C012_A02_SY09_P06_S03_06DAS.mp4', 'C012_A02_SY09_P06_S04_01DBS.mp4', 'C012_A02_SY09_P06_S04_02DAS.mp4', 'C012_A02_SY09_P07_S51_05DBS.mp4', 'C012_A02_SY09_P09_S02_01DBS.mp4', 'C012_A02_SY09_P10_S01_03DAS.mp4', 'C012_A02_SY09_P10_S02_03DAS.mp4', 'C012_A03_SY02_P07_S62_01DAS.mp4', 'C012_

In [13]:
# import random
# import shutil

# # 소스 폴더 경로
# source_folder = './CA_Y_Image'

# # 대상 폴더 경로들
# folder1 = "./train/CA_Y"
# folder2 = "./valid/CA_Y"
# folder3 = "./test/CA_Y"

# # 대상 폴더들을 리스트로 저장
# target_folders = [folder1, folder2, folder3]

# # 소스 폴더 내 파일 목록 가져오기
# file_list = os.listdir(source_folder)

# # 파일들을 랜덤하게 섞기
# random.shuffle(file_list)

# # 파일들을 7:2:1 비율로 각각의 폴더로 이동
# total_files = len(file_list)
# num_files_folder1 = total_files * 7 // 10
# num_files_folder2 = total_files * 2 // 10
# num_files_folder3 = total_files - num_files_folder1 - num_files_folder2

# for i, folder in enumerate(target_folders):
#     num_files = [num_files_folder1, num_files_folder2, num_files_folder3][i]
#     for filename in file_list[:num_files]:
#         source_path = os.path.join(source_folder, filename)
#         target_path = os.path.join(folder, filename)
#         shutil.copy(source_path, target_path)
#     # 이미 이동한 파일은 제외
#     file_list = file_list[num_files:]

# print("파일 이동이 완료되었습니다.")

파일 이동이 완료되었습니다.


In [14]:
# # 소스 폴더 경로
# source_folder = './CA_N_Image'

# # 대상 폴더 경로들
# folder1 = "./train/CA_N"
# folder2 = "./valid/CA_N"
# folder3 = "./test/CA_N"

# # 대상 폴더들을 리스트로 저장
# target_folders = [folder1, folder2, folder3]

# # 소스 폴더 내 파일 목록 가져오기
# file_list = os.listdir(source_folder)

# # 파일들을 랜덤하게 섞기
# random.shuffle(file_list)

# # 파일들을 7:2:1 비율로 각각의 폴더로 이동
# total_files = len(file_list)
# num_files_folder1 = total_files * 7 // 10
# num_files_folder2 = total_files * 2 // 10
# num_files_folder3 = total_files - num_files_folder1 - num_files_folder2

# for i, folder in enumerate(target_folders):
#     num_files = [num_files_folder1, num_files_folder2, num_files_folder3][i]
#     for filename in file_list[:num_files]:
#         source_path = os.path.join(source_folder, filename)
#         target_path = os.path.join(folder, filename)
#         shutil.copy(source_path, target_path)
#     # 이미 이동한 파일은 제외
#     file_list = file_list[num_files:]

# print("파일 이동이 완료되었습니다.")

파일 이동이 완료되었습니다.
